こちらのnotebookを参考にテキストを文字列に変換して特徴抽出を行った。
https://www.guruguru.science/competitions/16/discussions/fb792c87-6bad-445d-aa34-b4118fc378c1/

# 必要ライブラリのインポート

In [ ]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 212 kB 55.0 MB/s 


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 33.6 MB/s 
     |████████████████████████████████| 596 kB 48.9 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import collections
import os
import random

import matplotlib.pyplot as plt
import nlp
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from transformers import AdamW, AutoModel, AutoTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/00_datascience/16_studentcup'

Mounted at /content/drive
/content/drive/MyDrive/00_datascience/16_studentcup


# seed / configの設定

In [ ]:
# seeds
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

seed_everything(SEED)

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    current_device = torch.cuda.current_device()
    print("Device:", torch.cuda.get_device_name(current_device))

Device: Tesla T4


In [ ]:
# config
data_dir = '/content/drive/MyDrive/00_datascience/16_studentcup/input'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
TRAIN_FILE = '/content/drive/MyDrive/00_datascience/16_studentcup/input/train.csv'
TEST_FILE = '/content/drive/MyDrive/00_datascience/16_studentcup/input/test.csv'
SUB_FILE = '/content/drive/MyDrive/00_datascience/16_studentcup/input/submit_sample.csv'
MODELS_DIR = "./models/"
MODEL_NAME = 'bert-base-uncased'
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 128
NUM_CLASSES = 4
EPOCHS = 5
NUM_SPLITS = 5

# データのインポート

In [ ]:
train = pd.read_csv(TRAIN_FILE)
test = pd.read_csv(TEST_FILE)
sub = pd.read_csv(SUB_FILE)

In [ ]:
train.head()

,id,description,jobflag
0,0,<li>Develop cutting-edge web applications that...,3
1,1,"<li> Designs and develops high quality, scalab...",3
2,2,<li>Functions as a point person for Network St...,4
3,3,"<li> Work on the technical design, development...",3
4,4,<li>Quantify the resources required for a task...,4


# signateのチュートリアルで行われているクリーニングの実装

In [ ]:
import re
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def cleaning(texts):
    clean_texts = []
    for text in texts:
        # htmlタグを削除
        text = remove_tag(text)
        #アルファベット以外をスペースに置き換え
        clean_punc = re.sub(r'[^a-zA-Z]', ' ', text)
        #単語長が3文字以下のものは削除する
        clean_short_tokenized = [word for word in clean_punc.split() if len(word) > 3]
        #ステミング
        clean_normalize = [stemmer.stem(word) for word in clean_short_tokenized]
        #単語同士をスペースでつなぎ, 文章に戻す
        clean_text = ' '.join(clean_normalize)
        clean_texts.append(clean_text)
    return clean_texts

def remove_tag(x):
    p = re.compile(r"<[^>]*?>")
    return p.sub('',x)

train_cleaned = train.copy()
test_cleaned = test.copy()

train_cleaned['description'] = cleaning(train_cleaned['description'])
test_cleaned['description'] = cleaning(test_cleaned['description'])

# Bertによる特徴抽出

In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers

from transformers import BertTokenizer
from tqdm import tqdm
tqdm.pandas()

In [ ]:
class BertSequenceVectorizer:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = 'bert-base-uncased'
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128


    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out.cpu().detach().numpy()# , pooled_out.cpu().detach().numpy() #[0][0].cpu().detach().numpy() # 0番目は [CLS] token, 768 dim の文章特徴量
        else:
            return seq_out[0][0].detach().numpy()

In [ ]:
BSV = BertSequenceVectorizer()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ここからtrain dataの抽出

In [ ]:
seq_out_train = train_cleaned['description'].progress_apply(lambda x: BSV.vectorize(x))

100%|██████████| 1516/1516 [00:56<00:00, 26.85it/s]


In [ ]:
#seq_out_train

0       [[[-0.62222606, -0.03265866, 0.08519325, -0.35...
1       [[[-0.43817464, 0.18455186, 0.11949502, 0.0990...
2       [[[-0.2526526, 0.032130834, 0.09057478, -0.076...
3       [[[-0.59416354, -0.08183746, -0.04562828, -0.0...
4       [[[-0.23138937, -0.20248935, 0.13103667, -0.32...
                              ...                        
1511    [[[-0.58336616, -0.27802747, 0.1513256, -0.300...
1512    [[[-0.67049474, -0.18662664, 0.1901421, 0.0253...
1513    [[[-0.202192, 0.058532503, 0.16587526, -0.0716...
1514    [[[-0.7774473, -0.23549636, 0.26006833, 0.0508...
1515    [[[-0.5688777, 0.14548399, 0.44608033, -0.0730...
Name: description, Length: 1516, dtype: object

In [ ]:
#len(train)

1516

In [ ]:
#seq_out_train[2].reshape(1, -1)

array([[-0.2526526 ,  0.03213083,  0.09057478, ..., -0.07960205,
        -0.17735066,  0.20047486]], dtype=float32)

In [ ]:
new_train = pd.DataFrame(seq_out_train[0].reshape(1, -1))

for i in range(1, len(train)):
  new_train = new_train.append(pd.DataFrame(seq_out_train[i].reshape(1, -1)), ignore_index=True)
  print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
new_train.head()

,0,1,2,3,4,5,6,7,8,9,...,393206,393207,393208,393209,393210,393211,393212,393213,393214,393215
0,-0.622226,-0.032659,0.085193,-0.354324,-0.456699,-0.251040,0.150931,-0.084210,-0.072143,-0.478697,...,0.339774,-0.149296,0.053360,0.091067,-0.129803,-0.079903,-0.074918,-0.211404,-0.062383,-0.017082
1,-0.438175,0.184552,0.119495,0.099002,-0.214067,-0.111808,0.141238,0.045338,-0.244920,-0.417283,...,0.245712,-0.221893,-0.102024,0.031020,-0.216300,-0.120548,-0.176547,-0.168646,-0.258874,-0.096353
2,-0.252653,0.032131,0.090575,-0.076676,-0.106620,-0.055287,0.128359,-0.091651,-0.215434,-0.529369,...,0.121098,-0.288806,-0.161246,-0.057608,-0.285218,-0.176488,-0.185534,-0.079602,-0.177351,0.200475
3,-0.594164,-0.081837,-0.045628,-0.059278,-0.147906,-0.130730,0.528820,0.203796,-0.320909,-0.598328,...,0.381880,-0.099521,-0.137941,-0.098368,0.004457,0.026128,-0.265113,-0.134209,-0.108654,-0.095727
4,-0.231389,-0.202489,0.131037,-0.321926,-0.135216,-0.230279,0.191065,0.167886,-0.157697,-0.513312,...,0.414437,-0.041587,-0.125827,-0.047276,-0.171541,-0.047545,-0.194639,-0.000516,0.052555,-0.080560


In [ ]:
new_train.to_csv('train_with_400000feats.csv')

ここからテストデータの抽出

In [ ]:
seq_out_test = test_cleaned['description'].progress_apply(lambda x: BSV.vectorize(x))

In [ ]:
new_test = pd.DataFrame(seq_out_test[0].reshape(1, -1))

for i in range(1, len(test)):
  new_test = new_test.append(pd.DataFrame(seq_out_test[i].reshape(1, -1)), ignore_index=True)
  print(i)

In [ ]:
new_test.to_csv('test_with_400000feats.csv')